# Training Model AI Generated Menu Morning Box

In [9]:
# Import Library

# Pandas and Numpy are used for data manipulation and preprocessing
import pandas as pd
import numpy as np

# OneHotEncoder and MultiLabelBinarizer for encoding categorical variables and splitting the data into training and testing sets
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# TensorFlow with Keras for building the neural network model
import tensorflow as tf
from tensorflow.keras import layers

In [15]:
# Load Dataset
data = pd.read_csv('Dataset.csv')

In [16]:
# Convert survey answers to numeric
data[['survey_1', 'survey_2', 'survey_3', 'survey_4', 'survey_5']] = data[[
    'survey_1','survey_2', 'survey_3', 'survey_4', 'survey_5']
    ].applymap(lambda x: 0 if x == 'A' else 1)


In [17]:
# One-hot encoding for allergy column
# The code below is for preprocessing the allergy column of the dataset

# Convert non-string values to empty strings and remove the square brackets and double quotes
data['survey_6'] = data['survey_6'].apply(lambda x: x.strip(
    '[]').replace('"', '').split(',') if isinstance(x, str) else '')

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Apply one-hot encoding to the allergy column using MultiLabelBinarizer
encoded_allergies = mlb.fit_transform(data['survey_6'])

# Create column names
allergy_columns = ['allergy_' + col for col in mlb.classes_]

# Convert the encoded allergies array to a DataFrame and concat
encoded_allergies_df = pd.DataFrame(encoded_allergies, columns=allergy_columns)
data = pd.concat([data, encoded_allergies_df], axis=1)


In [18]:
# Create target labels
target_columns = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
y = data[target_columns]


In [19]:
# Encode target labels
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(y).toarray()

In [21]:
# Split data into training and testing sets
X = data[['survey_1', 'survey_2', 'survey_3', 'survey_4', 'survey_5'] + allergy_columns].values
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [22]:
# Build and train the model
# Create a sequential model, which is a linear stack of layers
model = tf.keras.Sequential([
    # Add a dense layer with 64 neurons, ReLU activation function, and input shape matching the number of features in X_train
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    # Add another dense layer with 64 neurons and ReLU activation function
    layers.Dense(64, activation='relu'),
    # Add the output layer with neurons equal to the number of categories in y_encoded and a softmax activation function
    layers.Dense(y_encoded.shape[1], activation='softmax')
])


In [23]:
# Configure the model for training by specifying the optimizer, loss function, and metric to be used
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the training data (X_train, y_train), validating it against the test data (X_test, y_test)
# The training process will run for 20 epochs, with a batch size of 32 samples
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)


Epoch 1/20
1/1 [==============================] - 1s 591ms/step - loss: 13.7042 - accuracy: 0.0000e+00 - val_loss: 13.6999 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 22ms/step - loss: 13.6999 - accuracy: 0.0000e+00 - val_loss: 13.6959 - val_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 27ms/step - loss: 13.6959 - accuracy: 0.0000e+00 - val_loss: 13.6930 - val_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 27ms/step - loss: 13.6930 - accuracy: 0.0000e+00 - val_loss: 13.6904 - val_accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 0s 27ms/step - loss: 13.6904 - accuracy: 0.0000e+00 - val_loss: 13.6881 - val_accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 0s 29ms/step - loss: 13.6881 - accuracy: 0.0000e+00 - val_loss: 13.6864 - val_accuracy: 0.0000e+00
Epoch 7/20
1/1 [==============================] - 0s 29ms/step - loss: 13.6864 - accuracy: 0.0000e+00 - v

In [24]:
# Make predictions
sample_input = np.array([X_test[0]])  # Replace with your input data
predictions = model.predict(sample_input)
recommendations = encoder.inverse_transform(predictions)


1/1 [==============================] - 0s 57ms/step


In [26]:
# Print recommendations
day_names = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
recommended_meals = {day: meal for day, meal in zip(day_names, recommendations[0])}
print(recommended_meals)


{'monday': nan, 'tuesday': nan, 'wednesday': nan, 'thursday': nan, 'friday': nan, 'saturday': nan, 'sunday': nan}
